In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.retrievers import ParentDocumentRetriever
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# Re-Rankers
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

from dotenv import load_dotenv
import os

In [4]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [5]:
# Carregar modelos open AI - EMbedding e CHAT

embeddings_model = OpenAIEmbeddings(model='text-embedding-3-small')
llm = ChatOpenAI(model='gpt-3.5-turbo', max_tokens=300)

/var/folders/3d/sw03jzns4xz8x8zxh2ksmfhw0000gn/T/ipykernel_46119/4038359212.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings_model = OpenAIEmbeddings(model='text-embedding-3-small')


In [6]:
# Carregar o PDF

pdf_link = 'pdf-investimentos-RAG.pdf'
loader = PyPDFLoader(pdf_link)
pages = loader.load_and_split()

In [7]:
len(pages)

25

In [10]:
# Separar em chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True
)

chunks = text_splitter.split_documents(pages)

In [11]:
# Salvar os chunks no vector db

vector_db = Chroma(embedding_function=embeddings_model, persist_directory='naiveDB')

/var/folders/3d/sw03jzns4xz8x8zxh2ksmfhw0000gn/T/ipykernel_46119/262822525.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(embedding_function=embeddings_model, persist_directory='naiveDB')


In [12]:
# Carregar o DB
naive_retreiver = vector_db.as_retriever(search_kwargs={"k": 10})

In [ ]:
load_dotenv()

os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY')

In [20]:
rerank = CohereRerank(model='rerank-v3.5', top_n=3)

compressor_retriver = ContextualCompressionRetriever(
    base_compressor=rerank, # Compressor novo, que vai ser usado para re-rank
    base_retriever=naive_retreiver # Primeiro retriever, que vai ser usado para buscar os documentos
)

In [17]:
TEMPLATE = """
    Você é um especialista em investimentos. Responda a pergunta utilizando o contexto informado.
    
    Query: 
    {question}
    
    Context:
    {context}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [21]:
setup_retrival = RunnableParallel({
    "question": RunnablePassthrough(),
    "context": compressor_retriver
    })

output_parser = StrOutputParser()

compressor_retrieval_chain = setup_retrival | rag_prompt | llm | output_parser


In [23]:
compressor_retrieval_chain.invoke("quais são os tipos de investimentos?")

'Existem vários tipos de investimentos disponíveis, tais como ações, títulos públicos, fundos de investimento, imóveis, poupança, entre outros. Cada tipo de investimento possui características e riscos diferentes, por isso é importante avaliar suas necessidades e objetivos antes de escolher onde aplicar seu dinheiro. É recomendável buscar orientação de um especialista em investimentos para ajudar na tomada de decisão.'